In [ ]:
%cp /content/drive/MyDrive/Books.json.gz /content

In [ ]:
%cd /content

/content


In [ ]:
!gunzip /content/Books.json.gz

In [ ]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 23.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=9190ad7c9ac6ad203b64b822884a3c79f8b5772ccb2ef1386402d2436bf02ce8
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("Big-Data-Multiclass-Classification").getOrCreate()

In [ ]:
df = spark.read.json("/content/Books.json")

In [ ]:
df

DataFrame[asin: string, image: array<string>, overall: double, reviewText: string, reviewTime: string, reviewerID: string, reviewerName: string, style: struct<Color::string,Format::string,Package Quantity::string,Size::string,Style Name::string,Style::string>, summary: string, unixReviewTime: bigint, verified: boolean, vote: string]

In [ ]:
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- image: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- style: struct (nullable = true)
 |    |-- Color:: string (nullable = true)
 |    |-- Format:: string (nullable = true)
 |    |-- Package Quantity:: string (nullable = true)
 |    |-- Size:: string (nullable = true)
 |    |-- Style Name:: string (nullable = true)
 |    |-- Style:: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- vote: string (nullable = true)



In [ ]:
df.dtypes

[('asin', 'string'),
 ('image', 'array<string>'),
 ('overall', 'double'),
 ('reviewText', 'string'),
 ('reviewTime', 'string'),
 ('reviewerID', 'string'),
 ('reviewerName', 'string'),
 ('style',
  'struct<Color::string,Format::string,Package Quantity::string,Size::string,Style Name::string,Style::string>'),
 ('summary', 'string'),
 ('unixReviewTime', 'bigint'),
 ('verified', 'boolean'),
 ('vote', 'string')]

In [ ]:
df.schema

StructType([StructField('asin', StringType(), True), StructField('image', ArrayType(StringType(), True), True), StructField('overall', DoubleType(), True), StructField('reviewText', StringType(), True), StructField('reviewTime', StringType(), True), StructField('reviewerID', StringType(), True), StructField('reviewerName', StringType(), True), StructField('style', StructType([StructField('Color:', StringType(), True), StructField('Format:', StringType(), True), StructField('Package Quantity:', StringType(), True), StructField('Size:', StringType(), True), StructField('Style Name:', StringType(), True), StructField('Style:', StringType(), True)]), True), StructField('summary', StringType(), True), StructField('unixReviewTime', LongType(), True), StructField('verified', BooleanType(), True), StructField('vote', StringType(), True)])

In [ ]:
df.show()

+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|               style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|0001713353| null|    5.0|This book is a wi...|08 12, 2005|A1C6M8LCIX4M6M|            June Bug|{null,  Paperback...| Children's favorite|    1123804800|   false|null|
|0001713353| null|    5.0|The King, the Mic...|03 30, 2005|A1REUF3A1YCPHM|         TW Ervin II|{null,  Hardcover...|A story children ...|    1112140800|   false|null|
|0001713353| null|    5.0|My daughter got h...| 04 4, 2004| A1YRBRK2XM5D5|   Rebecca L. Menner|{null,  Hardcover...|          Third copy|    1081036800|   false|   5

In [ ]:
df.select(df.reviewText).show(truncate=121)

+-------------------------------------------------------------------------------------------------------------------------+
|                                                                                                               reviewText|
+-------------------------------------------------------------------------------------------------------------------------+
|              This book is a winner with both of my boys.  They really enjoy the pictures and the story.  It's a classic.|
|The King, the Mice and the Cheese by Nancy Gurney is an excellent children's book.  It is one that I well remember fro...|
|My daughter got her first copy from her great-grandmother (it had been my father's when he was a child). She loves it ...|
|I remember this book from when I was a child about 30 years ago and what I most remember was how wonderful the express...|
|Just as I remembered it, one of my favorites from childhood! Great condition, very happy to have this to share with my...|
|       

In [ ]:
df.select(df["overall"]).show()

+-------+
|overall|
+-------+
|    5.0|
|    5.0|
|    5.0|
|    5.0|
|    5.0|
|    5.0|
|    5.0|
|    5.0|
|    5.0|
|    5.0|
|    5.0|
|    5.0|
|    5.0|
|    4.0|
|    5.0|
|    5.0|
|    5.0|
|    5.0|
|    5.0|
|    5.0|
+-------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import col

In [ ]:
df.select(col("summary")).show(truncate=200)

+---------------------------------------------------------------------------------------+
|                                                                                summary|
+---------------------------------------------------------------------------------------+
|                                                                    Children's favorite|
|                                              A story children will love and learn from|
|                                                                             Third copy|
|                                                                 Graphically Wonderful!|
|                                  Great condition, very happy to have this to share ...|
|                                                                             Five Stars|
|                                                                             Five Stars|
|           Got a special part of my childhood back aand a sharing it with my Grandkids!|
|         

In [ ]:
df.select("summary").show(truncate=200)

+---------------------------------------------------------------------------------------+
|                                                                                summary|
+---------------------------------------------------------------------------------------+
|                                                                    Children's favorite|
|                                              A story children will love and learn from|
|                                                                             Third copy|
|                                                                 Graphically Wonderful!|
|                                  Great condition, very happy to have this to share ...|
|                                                                             Five Stars|
|                                                                             Five Stars|
|           Got a special part of my childhood back aand a sharing it with my Grandkids!|
|         

In [ ]:
from pyspark.sql.functions import split

In [ ]:
df.select(split(df.reviewText," ")).show(truncate=121)

+-------------------------------------------------------------------------------------------------------------------------+
|                                                                                                 split(reviewText,  , -1)|
+-------------------------------------------------------------------------------------------------------------------------+
|[This, book, is, a, winner, with, both, of, my, boys., , They, really, enjoy, the, pictures, and, the, story., , It's,...|
|[The, King,, the, Mice, and, the, Cheese, by, Nancy, Gurney, is, an, excellent, children's, book., , It, is, one, that...|
|[My, daughter, got, her, first, copy, from, her, great-grandmother, (it, had, been, my, father's, when, he, was, a, ch...|
|[I, remember, this, book, from, when, I, was, a, child, about, 30, years, ago, and, what, I, most, remember, was, how,...|
|[Just, as, I, remembered, it,, one, of, my, favorites, from, childhood!, Great, condition,, very, happy, to, have, thi...|
|       

In [ ]:
df.select(split(df.reviewText," ",-1).alias("Tokenized Text")).show(truncate=121)

+-------------------------------------------------------------------------------------------------------------------------+
|                                                                                                           Tokenized Text|
+-------------------------------------------------------------------------------------------------------------------------+
|[This, book, is, a, winner, with, both, of, my, boys., , They, really, enjoy, the, pictures, and, the, story., , It's,...|
|[The, King,, the, Mice, and, the, Cheese, by, Nancy, Gurney, is, an, excellent, children's, book., , It, is, one, that...|
|[My, daughter, got, her, first, copy, from, her, great-grandmother, (it, had, been, my, father's, when, he, was, a, ch...|
|[I, remember, this, book, from, when, I, was, a, child, about, 30, years, ago, and, what, I, most, remember, was, how,...|
|[Just, as, I, remembered, it,, one, of, my, favorites, from, childhood!, Great, condition,, very, happy, to, have, thi...|
|       

In [ ]:
tokenized_text = df.select(split(df.reviewText," ",-1))
tokenized_text = tokenized_text.withColumnRenamed("split(reviewText,  , -1)","Tokenized Text")

In [ ]:
tokenized_text.show(truncate=121)

+-------------------------------------------------------------------------------------------------------------------------+
|                                                                                                           Tokenized Text|
+-------------------------------------------------------------------------------------------------------------------------+
|[This, book, is, a, winner, with, both, of, my, boys., , They, really, enjoy, the, pictures, and, the, story., , It's,...|
|[The, King,, the, Mice, and, the, Cheese, by, Nancy, Gurney, is, an, excellent, children's, book., , It, is, one, that...|
|[My, daughter, got, her, first, copy, from, her, great-grandmother, (it, had, been, my, father's, when, he, was, a, ch...|
|[I, remember, this, book, from, when, I, was, a, child, about, 30, years, ago, and, what, I, most, remember, was, how,...|
|[Just, as, I, remembered, it,, one, of, my, favorites, from, childhood!, Great, condition,, very, happy, to, have, thi...|
|       

In [ ]:
df.show()

+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|               style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|0001713353| null|    5.0|This book is a wi...|08 12, 2005|A1C6M8LCIX4M6M|            June Bug|{null,  Paperback...| Children's favorite|    1123804800|   false|null|
|0001713353| null|    5.0|The King, the Mic...|03 30, 2005|A1REUF3A1YCPHM|         TW Ervin II|{null,  Hardcover...|A story children ...|    1112140800|   false|null|
|0001713353| null|    5.0|My daughter got h...| 04 4, 2004| A1YRBRK2XM5D5|   Rebecca L. Menner|{null,  Hardcover...|          Third copy|    1081036800|   false|   5

In [ ]:
df.filter(df["overall"] == 5.0).show()

+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|               style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|0001713353| null|    5.0|This book is a wi...|08 12, 2005|A1C6M8LCIX4M6M|            June Bug|{null,  Paperback...| Children's favorite|    1123804800|   false|null|
|0001713353| null|    5.0|The King, the Mic...|03 30, 2005|A1REUF3A1YCPHM|         TW Ervin II|{null,  Hardcover...|A story children ...|    1112140800|   false|null|
|0001713353| null|    5.0|My daughter got h...| 04 4, 2004| A1YRBRK2XM5D5|   Rebecca L. Menner|{null,  Hardcover...|          Third copy|    1081036800|   false|   5

In [ ]:
df.where(df["overall"] == 5.0).show()

+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|               style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|0001713353| null|    5.0|This book is a wi...|08 12, 2005|A1C6M8LCIX4M6M|            June Bug|{null,  Paperback...| Children's favorite|    1123804800|   false|null|
|0001713353| null|    5.0|The King, the Mic...|03 30, 2005|A1REUF3A1YCPHM|         TW Ervin II|{null,  Hardcover...|A story children ...|    1112140800|   false|null|
|0001713353| null|    5.0|My daughter got h...| 04 4, 2004| A1YRBRK2XM5D5|   Rebecca L. Menner|{null,  Hardcover...|          Third copy|    1081036800|   false|   5

In [ ]:
df.where(df["overall"].isin(5.0)).show()

+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|               style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|0001713353| null|    5.0|This book is a wi...|08 12, 2005|A1C6M8LCIX4M6M|            June Bug|{null,  Paperback...| Children's favorite|    1123804800|   false|null|
|0001713353| null|    5.0|The King, the Mic...|03 30, 2005|A1REUF3A1YCPHM|         TW Ervin II|{null,  Hardcover...|A story children ...|    1112140800|   false|null|
|0001713353| null|    5.0|My daughter got h...| 04 4, 2004| A1YRBRK2XM5D5|   Rebecca L. Menner|{null,  Hardcover...|          Third copy|    1081036800|   false|   5

In [ ]:
grouped_data = df.groupby(df["overall"])

In [ ]:
grouped_data.count().show()

+-------+--------+
|overall|   count|
+-------+--------+
|    1.0| 2088314|
|    4.0| 9557650|
|    3.0| 3835225|
|    2.0| 1850688|
|    5.0|33979736|
|    0.0|       8|
+-------+--------+



In [ ]:
grouped_data = df.groupBy(df["overall"])

In [ ]:
grouped_data.count().show()

+-------+--------+
|overall|   count|
+-------+--------+
|    1.0| 2088314|
|    4.0| 9557650|
|    3.0| 3835225|
|    2.0| 1850688|
|    5.0|33979736|
|    0.0|       8|
+-------+--------+



#**```coalesce()```** to merge partitions into desired less number of partitions. 

In [ ]:
classification_df = df.select(df["overall"],df["reviewText"])

In [ ]:
classification_df

DataFrame[overall: double, reviewText: string]

In [ ]:
classification_df.columns

['overall', 'reviewText']

In [ ]:
cols = df.columns

In [ ]:
cols

['asin',
 'image',
 'overall',
 'reviewText',
 'reviewTime',
 'reviewerID',
 'reviewerName',
 'style',
 'summary',
 'unixReviewTime',
 'verified',
 'vote']

In [ ]:
classification_df = df.drop(*(cols[0:2]+cols[4:]))

In [ ]:
classification_df.columns

['overall', 'reviewText']

In [ ]:
classification_df = classification_df.select("reviewText","overall",
                         col("overall").cast("int").alias("overall_ratings_int"))

In [ ]:
classification_df

DataFrame[reviewText: string, overall: double, overall_ratings_int: int]

In [ ]:
classification_df.show()

+--------------------+-------+-------------------+
|          reviewText|overall|overall_ratings_int|
+--------------------+-------+-------------------+
|This book is a wi...|    5.0|                  5|
|The King, the Mic...|    5.0|                  5|
|My daughter got h...|    5.0|                  5|
|I remember this b...|    5.0|                  5|
|Just as I remembe...|    5.0|                  5|
|It is a very cute...|    5.0|                  5|
|  The kids loved it!|    5.0|                  5|
|I was just so hap...|    5.0|                  5|
|      good comdition|    5.0|                  5|
|My students (3 & ...|    5.0|                  5|
|A thought-provoki...|    5.0|                  5|
|My favorite book ...|    5.0|                  5|
|             LOVE IT|    5.0|                  5|
|Great story. I lo...|    4.0|                  4|
|This is a cute st...|    5.0|                  5|
|            Perfect!|    5.0|                  5|
|              Great!|    5.0| 

In [ ]:
classification_df = df.drop(*(cols[0:2]+cols[4:]))

In [ ]:
classification_df

DataFrame[overall: double, reviewText: string]

In [ ]:
classification_df = classification_df.select("reviewText","overall",
                                             col("overall")
                                             .cast("int").alias("overall"))

In [ ]:
classification_df

DataFrame[reviewText: string, overall: double, overall: int]

In [ ]:
classification_df = df.drop(*(cols[0:2]+cols[4:]))

In [ ]:
classification_df = classification_df.select("reviewText","overall",
                                             col("overall")
                                             .cast("int")
                                             .alias("overall_ratings_int")).drop("overall")

In [ ]:
classification_df

DataFrame[reviewText: string, overall_ratings_int: int]

In [ ]:
classification_df = df.drop(*(cols[0:2]+cols[4:]))

In [ ]:
classification_df = classification_df.withColumns({"overall_ratings_int":col("overall")
                                                  .cast("int")}).drop("overall")

In [ ]:
classification_df

DataFrame[reviewText: string, overall_ratings_int: int]

In [ ]:
df

DataFrame[asin: string, image: array<string>, overall: double, reviewText: string, reviewTime: string, reviewerID: string, reviewerName: string, style: struct<Color::string,Format::string,Package Quantity::string,Size::string,Style Name::string,Style::string>, summary: string, unixReviewTime: bigint, verified: boolean, vote: string]

In [ ]:
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- image: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- style: struct (nullable = true)
 |    |-- Color:: string (nullable = true)
 |    |-- Format:: string (nullable = true)
 |    |-- Package Quantity:: string (nullable = true)
 |    |-- Size:: string (nullable = true)
 |    |-- Style Name:: string (nullable = true)
 |    |-- Style:: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- vote: string (nullable = true)



In [ ]:
cols

['asin',
 'image',
 'overall',
 'reviewText',
 'reviewTime',
 'reviewerID',
 'reviewerName',
 'style',
 'summary',
 'unixReviewTime',
 'verified',
 'vote']

In [ ]:
summarization_df = df.drop(*(cols[0:8]+[cols[9]]))

In [ ]:
summarization_df

DataFrame[summary: string, verified: boolean, vote: string]

In [ ]:
summarization_df.show()

+--------------------+--------+----+
|             summary|verified|vote|
+--------------------+--------+----+
| Children's favorite|   false|null|
|A story children ...|   false|null|
|          Third copy|   false|   5|
|Graphically Wonde...|   false|null|
|Great condition, ...|    true|null|
|          Five Stars|    true|null|
|          Five Stars|    true|null|
|Got a special par...|    true|null|
|          Five Stars|    true|null|
|          Five Stars|    true|null|
|          Five Stars|    true|null|
|My favorite book ...|    true|null|
|          Five Stars|   false|null|
|Great story. I lo...|    true|null|
|This is a cute st...|    true|null|
|          Five Stars|    true|null|
|          Five Stars|    true|null|
|This was my 51 ye...|    true|null|
|A must have dr Se...|    true|null|
|      A true classic|    true|null|
+--------------------+--------+----+
only showing top 20 rows



In [ ]:
default_inner_join_df = classification_df.join(summarization_df,
                       on=(summarization_df["verified"]=="true"))

In [ ]:
default_inner_join_df.show()

+--------------------+-------------------+--------------------+--------+----+
|          reviewText|overall_ratings_int|             summary|verified|vote|
+--------------------+-------------------+--------------------+--------+----+
|This book is a wi...|                  5|Great condition, ...|    true|null|
|This book is a wi...|                  5|          Five Stars|    true|null|
|This book is a wi...|                  5|          Five Stars|    true|null|
|This book is a wi...|                  5|Got a special par...|    true|null|
|This book is a wi...|                  5|          Five Stars|    true|null|
|This book is a wi...|                  5|          Five Stars|    true|null|
|This book is a wi...|                  5|          Five Stars|    true|null|
|This book is a wi...|                  5|My favorite book ...|    true|null|
|This book is a wi...|                  5|Great story. I lo...|    true|null|
|This book is a wi...|                  5|This is a cute st...| 

In [ ]:
default_inner_join_df = summarization_df.join(classification_df,
                                              on=(summarization_df["verified"] == "true"))

In [ ]:
default_inner_join_df.show()

+--------------------+--------+----+--------------------+-------------------+
|             summary|verified|vote|          reviewText|overall_ratings_int|
+--------------------+--------+----+--------------------+-------------------+
|Great condition, ...|    true|null|This book is a wi...|                  5|
|Great condition, ...|    true|null|The King, the Mic...|                  5|
|Great condition, ...|    true|null|My daughter got h...|                  5|
|Great condition, ...|    true|null|I remember this b...|                  5|
|Great condition, ...|    true|null|Just as I remembe...|                  5|
|Great condition, ...|    true|null|It is a very cute...|                  5|
|Great condition, ...|    true|null|  The kids loved it!|                  5|
|Great condition, ...|    true|null|I was just so hap...|                  5|
|Great condition, ...|    true|null|      good comdition|                  5|
|Great condition, ...|    true|null|My students (3 & ...|       

In [ ]:
left_or_left_outer_join_df = classification_df.join(summarization_df,
                                                    on=(summarization_df["verified"]=="true"))

In [ ]:
left_or_left_outer_join_df.show()

+--------------------+-------------------+--------------------+--------+----+
|          reviewText|overall_ratings_int|             summary|verified|vote|
+--------------------+-------------------+--------------------+--------+----+
|This book is a wi...|                  5|Great condition, ...|    true|null|
|This book is a wi...|                  5|          Five Stars|    true|null|
|This book is a wi...|                  5|          Five Stars|    true|null|
|This book is a wi...|                  5|Got a special par...|    true|null|
|This book is a wi...|                  5|          Five Stars|    true|null|
|This book is a wi...|                  5|          Five Stars|    true|null|
|This book is a wi...|                  5|          Five Stars|    true|null|
|This book is a wi...|                  5|My favorite book ...|    true|null|
|This book is a wi...|                  5|Great story. I lo...|    true|null|
|This book is a wi...|                  5|This is a cute st...| 

In [ ]:
left_or_left_outer_join_df.where(left_or_left_outer_join_df["summary"] == "null").head()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored